<a href="https://colab.research.google.com/github/trista-paul/DS-Unit-4-Sprint-2-NLP/blob/master/Trista_DS42SC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 2 Sprint Challenge*

# Natural Language Processing

**Part 1 - Working with Text Data**
Use Python string methods remove irregular whitespace from the following string:

In [102]:
whitespace_string = "\n\n  This is a    string   that has  \n a lot of  extra \n   whitespace.   "
whitespace_string = whitespace_string.split(" ")
whitespace_string = filter(None, whitespace_string)
whitespace_string = " ".join(whitespace_string)
whitespace_string = whitespace_string.split('\n')
newlist = []
for word in whitespace_string:
  word = word.lstrip()
  newlist.append(word)
whitespace_string = filter(None, newlist)
whitespace_string = "".join(whitespace_string)

print(whitespace_string)

This is a string that has a lot of extra whitespace.


In [0]:
 ##### Your Code Here #####

### Use Regular Expressions to take the dates in the following .txt file and put them into a dataframe with columns for:

[RegEx dates.txt](https://raw.githubusercontent.com/ryanleeallred/datasets/master/dates.txt)

- Day
- Month
- Year


In [23]:
import pandas as pd
dates = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/dates.txt', header=None)
dates

dates['year'] = dates[1]
dates['month'] = dates[0].str.extract('(^\w+)', expand=True)
dates['day'] = dates[0].str.extract('(\w+$)', expand = True)

dates = dates.drop(columns=[0, 1])

dates

,year,month,day
0,2015,March,8
1,2015,March,15
2,2015,March,22
3,2015,March,29
4,2015,April,5
5,2015,April,12
6,2015,April,19
7,2015,April,26
8,2015,May,3
9,2015,May,10


# Part 2 - Bag of Words 

### Use the twitter sentiment analysis dataset found at this link for the remainder of the Sprint Challenge:

[Twitter Sentiment Analysis Dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv)

 ### Clean and tokenize the documents ensuring the following properties of the text:

1) Text should be lowercase.

2) Stopwords should be removed.

3) Punctuation should be removed.

4) Tweets should be tokenized at the word level. 

(The above don't necessarily need to be completed in that specific order.)

### Output some cleaned tweets so that we can see that you made all of the above changes.


In [63]:
tweets = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv')
tweets.head()

,Sentiment,SentimentText
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...


In [26]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

import string
from string import punctuation
table = str.maketrans('', '', punctuation)
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
def process_text(text):
    #sep by word
    words = word_tokenize(text)

    #lowercase
    words = [word.lower() for word in words]

    #remove non alphanumeric characters
    words = [word.translate(table) for word in words]
    words = [word for word in words if word.isalpha()]
    
    #remove stopwords
    words = [word for word in words if not word in stop_words]
    
    #lemmatize (remove prefixes and suffixes)
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    #final reformat
    words = " ".join(words)
    
    return words

tweets['SentimentText'] = tweets['SentimentText'].apply(process_text)

In [57]:
tweets.head()

,Sentiment,SentimentText
0,0,sad apl friend
1,0,missed new moon trailer
2,1,omg already
3,0,omgaga im sooo im gunna cry dentist since supo...
4,0,think mi bf cheating tt


### How should TF-IDF scores be interpreted? How are they calculated?

**Tf-Idf stands for term frequency - inverse document frequency**, and its formula multiplies these two named variables. In the context of a dataframe, a 'document' is an individual row of text.

**Term frequency** divides the number of times a selected word appears to the total words in the selected row.

**Inverse document frequency** divides (and then logs) the total number of rows with the number of rows where the word appears. 

Making a dataframe of words and their tf-idf score per row is a powerful way to **evaluate unique and unusual words in the data**. The largest tf-idfs appear frequently in the selected row but infrequently across all rows.

# Part 3 - Document Classification

1) Use Train_Test_Split to create train and test datasets.

2) Vectorize the tokenized documents using your choice of vectorization method. 

 - Stretch goal: Use both of the methods that we talked about in class.

3) Create a vocabulary using the X_train dataset and transform both your X_train and X_test data using that vocabulary.

4) Use your choice of binary classification algorithm to train and evaluate your model's accuracy. Report both train and test accuracies.

 - Stretch goal: Use an error metric other than accuracy and implement/evaluate multiple classifiers.
 - Stretch goal: Track your results in a DataFrmae and produce a visualization of the results



In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from xgboost.sklearn import XGBClassifier

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [0]:
X = tweets['SentimentText']
y = tweets['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [0]:
vectorizer = TfidfVectorizer(max_features=2000, ngram_range=(1,2),
                             min_df = 5, max_df = .80,
                             stop_words = 'english')
vectorizer.fit(X_train)
train_word_counts = vectorizer.transform(X_train)
X_train_vectorized = pd.DataFrame(train_word_counts.toarray(),
                                  columns=vectorizer.get_feature_names())

In [67]:
model = XGBClassifier(max_depth = 2,
                      learning_rate = 0.1,
                      verbosity = 0,
                      n_jobs = -1,
                      random_state = 0)

model.fit(X_train_vectorized, y_train)

train_predict = model.predict(X_train_vectorized)
train_accuracy = accuracy_score(y_train, train_predict)
print(train_accuracy)

train_predict_proba = model.predict_proba(X_train_vectorized)[:, 1]
train_roc_auc = roc_auc_score(y_train, train_predict_proba)
print(train_roc_auc)

0.6692050520059435
0.7294788788632267


In [68]:
vectorizer.fit(X_test)
test_word_counts = vectorizer.transform(X_test)
X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

model.fit(X_test_vectorized, y_test)

test_predict = model.predict(X_test_vectorized)
test_accuracy = accuracy_score(y_test, test_predict)
print(test_accuracy)

test_predict_proba = model.predict_proba(X_test_vectorized)[:, 1]
test_roc_auc = roc_auc_score(y_test, test_predict_proba)
print(test_roc_auc)

0.6698669866986698
0.7406497269636835


In [73]:
bayes = MultinomialNB()
bayes.fit(X_train_vectorized, y_train)

train_predict = bayes.predict(X_train_vectorized)
train_accuracy2 = accuracy_score(y_train, train_predict)
print(train_accuracy2)

train_predict_proba = bayes.predict_proba(X_train_vectorized)[:, 1]
train_roc_auc2 = roc_auc_score(y_train, train_predict_proba)
print(train_roc_auc2)

0.7474997142530575
0.8255992540055059


In [74]:
bayes.fit(X_test_vectorized, y_test)

test_predict = bayes.predict(X_test_vectorized)
test_accuracy2 = accuracy_score(y_test, test_predict)
print(test_accuracy2)

test_predict_proba = bayes.predict_proba(X_test_vectorized)[:, 1]
test_roc_auc2 = roc_auc_score(y_test, test_predict_proba)
print(test_roc_auc2)

0.759042570923759
0.8406857373569174


In [0]:
count = CountVectorizer(max_features=2000, ngram_range=(1,2),
                             min_df = 5, max_df = .80,
                             stop_words = 'english')
vectorizer.fit(X_train)
train_word_counts = vectorizer.transform(X_train)
X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

vectorizer.fit(X_test)
test_word_counts = vectorizer.transform(X_test)
X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

In [76]:
model.fit(X_train_vectorized, y_train)

train_predict = model.predict(X_train_vectorized)
train_accuracy3 = accuracy_score(y_train, train_predict)
print(train_accuracy3)

train_predict_proba = model.predict_proba(X_train_vectorized)[:, 1]
train_roc_auc3 = roc_auc_score(y_train, train_predict_proba)
print(train_roc_auc3)

model.fit(X_test_vectorized, y_test)

test_predict = model.predict(X_test_vectorized)
test_accuracy3 = accuracy_score(y_test, test_predict)
print(test_accuracy3)

test_predict_proba = model.predict_proba(X_test_vectorized)[:, 1]
test_roc_auc3 = roc_auc_score(y_test, test_predict_proba)
print(test_roc_auc3)

0.6692050520059435
0.7294788788632267
0.6698669866986698
0.7406497269636835


In [77]:
bayes.fit(X_train_vectorized, y_train)

train_predict = bayes.predict(X_train_vectorized)
train_accuracy4 = accuracy_score(y_train, train_predict)
print(train_accuracy4)

train_predict_proba = bayes.predict_proba(X_train_vectorized)[:, 1]
train_roc_auc4 = roc_auc_score(y_train, train_predict_proba)
print(train_roc_auc4)

bayes.fit(X_test_vectorized, y_test)

test_predict = bayes.predict(X_test_vectorized)
test_accuracy4 = accuracy_score(y_test, test_predict)
print(test_accuracy4)

test_predict_proba = bayes.predict_proba(X_test_vectorized)[:, 1]
test_roc_auc4 = roc_auc_score(y_test, test_predict_proba)
print(test_roc_auc4)

0.7474997142530575
0.8255992540055059
0.759042570923759
0.8406857373569174


In [79]:
row1 = ['tfidf', 'xgboost', train_accuracy, test_accuracy, train_roc_auc, test_roc_auc]
row2 = ['tfidf', 'bayes', train_accuracy2, test_accuracy2, train_roc_auc2, test_roc_auc2]
row3 = ['count', 'xgboost', train_accuracy3, test_accuracy3, train_roc_auc3, test_roc_auc3]
row4 = ['count', 'bayes', train_accuracy4, test_accuracy4, train_roc_auc4, test_roc_auc4]

data = [row1, row2, row3, row4]
model_comp = pd.DataFrame(data, columns = ['vectorizer', 'model',
                                           'train_accuracy', 'test_accuracy',
                                           'train_roc_auc', 'test_roc_auc'])

model_comp

,vectorizer,model,train_accuracy,test_accuracy,train_roc_auc,test_roc_auc
0,tfidf,xgboost,0.669205,0.669867,0.729479,0.740650
1,tfidf,bayes,0.747500,0.759043,0.825599,0.840686
2,count,xgboost,0.669205,0.669867,0.729479,0.740650
3,count,bayes,0.747500,0.759043,0.825599,0.840686


# Part 4 -  Word2Vec

1) Fit a Word2Vec model on your cleaned/tokenized twitter dataset. 

2) Display the 10 words that are most similar to the word "twitter"

In [0]:
import gensim
from gensim.models.word2vec import Word2Vec

In [0]:
#retokenize
tweets['SentimentText'] = tweets['SentimentText'].apply(word_tokenize)

In [59]:
tweets.head()

,Sentiment,SentimentText
0,0,"[sad, apl, friend]"
1,0,"[missed, new, moon, trailer]"
2,1,"[omg, already]"
3,0,"[omgaga, im, sooo, im, gunna, cry, dentist, si..."
4,0,"[think, mi, bf, cheating, tt]"


In [0]:
word = Word2Vec(tweets['SentimentText'])

In [61]:
word.wv.vocab.keys()

dict_keys(['sad', 'friend', 'missed', 'new', 'moon', 'trailer', 'omg', 'already', 'im', 'sooo', 'gunna', 'cry', 'dentist', 'since', 'get', 'crown', 'put', 'think', 'mi', 'bf', 'cheating', 'tt', 'worry', 'much', 'chillin', 'sunny', 'work', 'tomorrow', 'tv', 'tonight', 'handed', 'uniform', 'today', 'miss', 'hmmmm', 'wonder', 'number', 'must', 'positive', 'thanks', 'hater', 'face', 'day', 'weekend', 'sucked', 'far', 'jb', 'isnt', 'showing', 'australia', 'ok', 'thats', 'win', 'lt', 'way', 'feel', 'right', 'man', 'completely', 'useless', 'rt', 'funny', 'twitter', 'http', 'feeling', 'strangely', 'fine', 'gon', 'na', 'go', 'listen', 'celebrate', 'huge', 'roll', 'thunder', 'scary', 'cut', 'beard', 'growing', 'well', 'year', 'start', 'happy', 'meantime', 'iran', 'one', 'see', 'cause', 'else', 'following', 'pretty', 'awesome', 'level', 'writing', 'massive', 'blog', 'tweet', 'myspace', 'comp', 'shut', 'lost', 'lay', 'position', 'headed', 'pull', 'golf', 'place', 'something', 'yeah', 'boring', 'wh

In [62]:
word.wv.most_similar('twitter')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('message', 0.8484869003295898),
 ('sent', 0.8384042382240295),
 ('facebook', 0.834929347038269),
 ('dm', 0.8238285779953003),
 ('email', 0.8222038745880127),
 ('following', 0.8173774480819702),
 ('page', 0.8148053884506226),
 ('reply', 0.80271977186203),
 ('account', 0.8025115728378296),
 ('update', 0.7990567684173584)]